In [1]:
#Welcome to the Worlds of Belief Project!
#Below is the code for both a multivariable Deep Neural Network (DNN)
#analyzing the joint relationship between 6 belief predictors on democracy.
import os
print(os.getcwd())



C:\Users\ryant\tensorflow_env
